### Import relevant matrices

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse

t2 = pd.read_csv('/path/to/T2.csv')
t5 = pd.read_csv('/path/to/T5.csv')
t6_cont = pd.read_csv('/path/to/T6_cont.csv')
x = sparse.load_npz('/path/to/cls_T11_x.npz')
x = x.toarray()
y = sparse.load_npz('/path/to/cls_T10_y.npz')
y = y.toarray()
folds = np.load('/path/to/cls_T11_fold_vector.npy')

Check that t6_cont and the X, y and fold vectors match

In [ ]:
assert len(folds) == len(t6_cont) == len(x) == len(y)
assert np.sum(np.abs(t6_cont.fold_id.astype(float).to_numpy() - folds)) == 0

### Define lookup function to map from X and Y arrays back to SMILES

In [ ]:
def lookup(idx, t6_cont, t5, t2):
    """
    Lookup function to go from arrays generated via MELLODDY-Tuner back to input SMILES.
    
    Path goes like this:
    T6_cont (descriptor vector ID) -> T5 (descriptor vector ID, input compound ID) -> T2 (input compound ID, SMILES)

    T6_cont already includes some preprocessing like dropping compounds with no measurements, or the ones that
    failed standardization. 
    Crucially, it is also post-aggregation, meaning, it can be that for a given descriptor vector ID, 
    multiple compound IDs can be found. In that case, we pick the first (they should be all extremely 
    similar if they got the same ECFP/RDKIT/...)
    """
    map_1 = t6_cont.iloc[idx]['descriptor_vector_id']
    map_2 = t5.loc[t5.descriptor_vector_id == map_1]['input_compound_id'].iloc[0]
    map_3 = t2.loc[t2.input_compound_id == map_2]["smiles"].iloc[0]

    return map_3

### Get all SMILES 

In [ ]:
from tqdm import tqdm

smi = []
for i in tqdm(range(len(t6_cont)), desc="Processing"):
    smi.append(lookup(i, t6_cont, t5, t2))

In [ ]:
assert len(smi) == len(x) == len(y)

Run a few tests to see if descriptors indeed match the output of MELLODDY-Tuner (Careful: this only works if you point to the RDKIT output in the first cell)

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
import random
rdkit_all_descriptors = [x[0] for x in Descriptors._descList]
calc = MolecularDescriptorCalculator(rdkit_all_descriptors)

n_tests = 500           # edit this if you want to test more

random_indices = random.sample(range(len(smi)), n_tests)

mae = []
for i in random_indices:
    mol = Chem.MolFromSmiles(smi[i])
    out = calc.CalcDescriptors(mol)
    mae.append(np.mean(np.abs(x[i] - out)))

In [ ]:
assert np.median(mae) == 0

In [ ]:
import matplotlib.pyplot as plt

plt.hist(mae, bins = 20)
plt.xlabel("MAE between lookup and MELLODDY-Tuner")
plt.title("MAE distribution")
plt.ylabel("Count")

### Format as a Dataframe for Chemprop CLI

In [ ]:
# Chemprop needs a SMILES column first, then all the others are labels
colnames = [f"MELLODDY_{x}" for x in range(y.shape[1])]
df = pd.DataFrame(data=y,
                  columns=colnames)
df["SMILES"] = smi
new_col_order = ["SMILES"] + colnames
df = df[new_col_order]

# Labels should be either 0 or 1, with NaN for missing values
# In sparsechem, negative labels instead are -1 and missing labels are 0
df = df.replace(0, np.nan)
df = df.replace(-1, 0)

### Create train and test splits

In [ ]:
test_id = 0

df["FOLD_ID"] = t6_cont.fold_id
test_set = df[df["FOLD_ID"] == test_id]
train_set = df[df["FOLD_ID"] != test_id]
train_set = train_set.drop("FOLD_ID", axis=1)
test_set = test_set.drop("FOLD_ID", axis=1)

print(f"Length of the training set: {len(train_set)}")
print(f"Length of the test set: {len(test_set)}")

### Save output

In [ ]:
train_set.to_csv('/path/to/train_set.csv')
test_set.to_csv('/path/to/train_set.csv')